# Module 10 - Programming Assignment


# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [16]:
from unification import parse, unification, unify 

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

You will be solving the problem from above. Here is the start state:

In [17]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [18]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [19]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

<a id="goal_found"></a>
## goal_found

Determine is the goal is found **Used by**: [forward_planner](#forward_planner)

* **state: List[str]:** The current state of the program
* **goal: List[str]:** The goal state of the program

* **Returns Bool:** Return boolean of is a match is made

In [20]:
def goal_found(state, goal):
    return all(item in state for item in goal)

In [21]:
assert goal_found(["(item Saw)"], ["(item Saw)"]) == True
assert goal_found(["(item any)"], ["(item any)"]) == True
assert goal_found(["(item any)"], ["(any any)"]) == False

<a id="can_unify"></a>
## can_unify

Bool test for posible unification of S-expressions **Used by**: [generate_allowed_actionsr](#generate_allowed_actions)


* **conditions List[]:** list of conditions S-expressions 
* **state List[]:** list of state S-expressions 
* **retuns bool:** bool of if the whole condition is met in the current state

In [22]:
def can_unify(conditions, state):
    check=[]
    result=[]
    for condition_item in conditions:
        for state_item in state:
            check.append(unify(state_item, condition_item) != False)
        result.append(any(check))
    return all(result)

In [23]:
assert can_unify(["(agent ?agent)","(place ?from)","(place ?to)","(at ?agent ?from)"],
                    ["(item Saw)", "(item Drill)", "(place Home)", "(place Store)", "(place Bank)", "(agent Me)", "(at Me Home)", "(at Drill Me)", "(at Saw Store)"]) ==True
assert can_unify(["(agent ?agent)","(place ?from)","(place ?to)","(at ?agent ?from)"],
                    ["(item Saw)","(item Drill)","(place Home)","(place Store)","(place Bank)","(agent Me)","(at Me Home)","(at Saw Store)","(at Drill Store)"])== True
assert can_unify(["(agent ?agent)","(place ?from)","(place ?to)","(at ?agent ?from)"],
                    ["(item Saw)","(item Drill)","(place Home)","(place Store)","(place Bank)","(notagent Me)","(at Me Home)","(at Saw Store)","(at Drill Store)"]) ==False

<a id="replace"></a>
## replace

replace variables in expression with sublist values **Used by**: [update](#update)

* **expression dict:** expression being updated 
* **variable dict:** sublist used for replacement 
* **returns dict:** updated expression 

In [24]:
def replace(expression, variable_sublist):
    for var, value in variable_sublist.items():
        expression = expression.replace(var, value)
    return expression

<a id="update"></a>
## update

provides updated values from sublist for action **Used by**: [generate_allowed_actions](#generate_allowed_actions)

* **action dict:** action to be updated
* **sublist dict:** variable to use for updated action
* **returns dict:**  updated action

In [25]:
def update(action, sublist):
    update_action = {
        'action': replace(action['action'], sublist),
        'add': [replace(effect, sublist) for effect in action['add']],
        'delete': [replace(effect, sublist) for effect in action['delete']]
    }
    return update_action

<a id="generate_allowed_actions"></a>
## generate_allowed_actions

find the allowed actions found by unification **Used by**: [generate_sucessor_states](#generate_sucessor_states)

* **current_state List[str]:** The current state of the planning algorithm 
* **actions dict:** The allowable actions in planning algorithm 
* **return List[]:** list of actions

In [26]:
def generate_allowed_actions(current_state, actions):
    applicable_actions = []
    for action in list(actions.values()):
        if can_unify(action['conditions'], current_state):
            sublist = {}
            for condition_item in action['conditions']:
                for state_item in current_state:
                    frame = unify(state_item, condition_item)
                    if frame is not False:
                        sublist.update(frame)
                if sublist:
                    applicable_actions.append(update(action, sublist))
    return applicable_actions

In [27]:
assert generate_allowed_actions(["(blah Saw)"], actions) == []


<a id="generate_sucessor_states"></a>
## generate_sucessor_states

updates the plan/successor states **Used by**: [forward_planner](#forward_planners)

* **current_state dict:** current state being evaluted 
* **actions dict:** allowed actions 
* **returns list[dict]:** updated state and plan

In [28]:
def generate_sucessor_states(current_state, actions):
    result=[]
    for action in generate_applicable_actions(current_state, actions):
        pass

In [29]:
def forward_planner( start_state, goal, actions, debug=False):
    frontier = []
    explored = []
    path = []
    frontier.append(start_state)
#     path.append(start)
    while len(frontier) != 0: # needs work
        current_state = frontier.pop()
        if goal_found(state, goal):
            explored.append(current_state)
#             path.append(current_state)
#             return explored
        children = generate_sucessor_states(current_state, actions)
        for child in children:
            if child not in explored + frontier:
                frontier.append(child)
        explored.append(current_state)
    return explored # change to return the real answer
    return []

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [30]:
plan = forward_planner( start_state, goal, actions)

NameError: name 'state' is not defined

In [31]:
for el in plan:
    print(el)

NameError: name 'plan' is not defined